In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import polars as pl
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!cat /kaggle/input/amazon-review-full-csv/amazon_review_full_csv/readme.txt

In [ ]:
!pip install tiktoken pyspark


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("PySpark_Tutorial") \
    .config("spark.driver.memory", "10g") \
    .config("spark.executor.memory", "10g") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/02 16:05:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, IntegerType
import tiktoken
import numpy as np

PATH_TRAIN_DATA = "/kaggle/input/amazon-review-full-csv/amazon_review_full_csv/train.csv"
PATH_TEST_DATA = "/kaggle/input/amazon-review-full-csv/amazon_review_full_csv/test.csv"
rating_encodings = [
    [1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1],
]


def as_udf(output_annotation):
    def wrapper(func):
        return udf(func, output_annotation)
    return wrapper

@as_udf(ArrayType(IntegerType()))
def encode_rating(rating: str):
    return rating_encodings[int(rating[-1]) - 1]

@as_udf(ArrayType(IntegerType()))
def encode_text(text):
    gpt2_encoder = tiktoken.get_encoding("gpt2")
    text = text[:-1]
    encoded_text = gpt2_encoder.encode(text)
    len_encoded_text = len(encoded_text)
    return encoded_text + (700 - len_encoded_text)*[0]


In [3]:
path = "/kaggle/input/amazon-review-full-csv/amazon_review_full_csv/train.csv"
df_raw = spark.read.csv(path, header=False, inferSchema=True, sep='","')
df = df_raw.withColumn("_c0_1", encode_rating(df_raw._c0))
df = df.withColumn("_c2_1", encode_text(df_raw._c2))
df = df.select(
    df._c0_1, 
    df._c2_1,
)
df.coalesce(1).write.parquet("train2.parquet")
del df_raw, df

In [4]:
data = spark.read.parquet("/kaggle/working/train2.parquet") 

In [7]:
data.offset(50_000).limit(50_000).show()

+---------------+--------------------+
|          _c0_1|               _c2_1|
+---------------+--------------------+
|[0, 1, 0, 0, 0]|[1212, 318, 517, ...|
|[0, 1, 0, 0, 0]|[3260, 3555, 1353...|
|[0, 0, 0, 0, 1]|[1212, 318, 257, ...|
|[0, 0, 0, 0, 1]|[40, 1101, 6655, ...|
|[0, 0, 1, 0, 0]|[40, 373, 11679, ...|
|[0, 0, 0, 0, 1]|[464, 2342, 318, ...|
|[0, 0, 0, 1, 0]|[11980, 12666, 50...|
|[0, 0, 1, 0, 0]|[464, 287, 6753, ...|
|[1, 0, 0, 0, 0]|[4711, 443, 1130,...|
|[1, 0, 0, 0, 0]|[40, 5839, 777, 7...|
|[0, 1, 0, 0, 0]|[20, 6, 20, 15931...|
|[0, 0, 1, 0, 0]|[40, 588, 262, 20...|
|[0, 0, 1, 0, 0]|[4711, 443, 1130,...|
|[0, 0, 1, 0, 0]|[27218, 443, 1130...|
|[0, 0, 1, 0, 0]|[21902, 13822, 30...|
|[0, 0, 0, 0, 1]|[1212, 1720, 318,...|
|[0, 1, 0, 0, 0]|[40, 1101, 642, 6...|
|[0, 1, 0, 0, 0]|[40, 1422, 470, 1...|
|[0, 0, 0, 1, 0]|[35422, 1068, 777...|
|[0, 0, 1, 0, 0]|[40, 588, 777, 44...|
+---------------+--------------------+
only showing top 20 rows



In [ ]:
import numpy as np 
def read(idx):
    b = data.offset(idx*50_000).limit(50_000)
    x = np.array(b.select(b._c2_1).collect())[:, 0, :]
    y = np.array(b.select(b._c0_1).collect())[:, 0, :]
    return x, y

In [ ]:
read(15)